<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>
<h1 align=center><font size = 8>Data Science Capstone Project</font></h1>

In [6]:
try:
    import pandas as pd
    import numpy as np
except:
    !conda install -c anaconda pandas numpy --yes 
    import pandas as pd
    import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
try:
    from geopy.geocoders import Nominatim # to convert address into latitude and longitude
except:
    !conda install -c conda-forge geopy --yes 
    from geopy.geocoders import Nominatim
try:
    import folium # plotting library
except:
    !conda install -c conda-forge folium --yes 
    import folium
from bs4 import BeautifulSoup
import urllib.request
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import geojson

## Read Zillow geodata

In [7]:
with open('e:/downloads/zillow-neighborhoods.geojson') as json_data:
    zillow_neighborhood_data = geojson.load(json_data)

In [62]:
neighborhood_data = zillow_neighborhood_data['features']
df = pd.DataFrame(columns=['city','name','regionid','geo_point_2d','county','state'])
for entry in neighborhood_data:
    df = df.append( entry['properties'],ignore_index=True)
df = df.rename(columns={'name': 'Neighborhood'})
df.head()

,city,Neighborhood,regionid,geo_point_2d,county,state
0,Nashville,The Woodlands,416247,"[36.036945803981524, -86.72383769726473]",Davidson,TN
1,Nashville,Historic Edgefield,276378,"[36.17232426720076, -86.75750615331977]",Davidson,TN
2,Nashville,Hickory Bend,416544,"[36.1536040985679, -86.645776908022]",Davidson,TN
3,Pflugerville,Mountain Creek,105704,"[30.429440318783104, -97.63082138134793]",Travis,TX
4,San Antonio,Hidden Forest,271424,"[29.585374370557354, -98.50674820335668]",Bexar,TX


Split up geo_point_2d into Latitude and Longitude

In [63]:
print(df.shape)
df = df.dropna(subset=['geo_point_2d'])
print(df.shape)
df2 = pd.DataFrame(df["geo_point_2d"].tolist(), columns=['Latitude','Longitude']) #pd.DataFrame(df['geo_point_2d'].tolist(),index=df.index)
df = pd.concat([df, df2], axis=1)
df.drop(['geo_point_2d','regionid'],axis=1,inplace=True)
df.head()

(17057, 6)
(17029, 6)


,city,Neighborhood,county,state,Latitude,Longitude
0,Nashville,The Woodlands,Davidson,TN,36.036946,-86.723838
1,Nashville,Historic Edgefield,Davidson,TN,36.172324,-86.757506
2,Nashville,Hickory Bend,Davidson,TN,36.153604,-86.645777
3,Pflugerville,Mountain Creek,Travis,TX,30.429440,-97.630821
4,San Antonio,Hidden Forest,Bexar,TX,29.585374,-98.506748


Extract Dallas-Fort Worth

In [64]:
dfw = df[df['city'].isin(['Dallas', 'Fort Worth'])]
print(dfw.shape)
dfw.head()

(259, 6)


,city,Neighborhood,county,state,Latitude,Longitude
5,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357
62,Fort Worth,Fairmount,Tarrant,TX,48.116166,-123.468081
250,Fort Worth,Willow Creek,Tarrant,TX,29.653767,-95.482293
254,Fort Worth,Shaw Clarke,Tarrant,TX,30.142715,-94.191578
303,Fort Worth,Carver Heights,Tarrant,TX,36.898830,-76.228158


## Create a map of DFW Neighborhoods

In [102]:
# create map - use Arlington for map lat/long
address = 'Arlington, TX'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Arlington are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Arlington are 32.701938999999996, -97.10562379033699.


In [106]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfw['Latitude'], dfw['Longitude'], dfw['city'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Get Neighborhood Venues

In [56]:
CLIENT_ID = 'YPF4NY2VJMYEMROHWYBYVI11IZKQ0H4EDECKGJR0XDYRWE4M' # your Foursquare ID
CLIENT_SECRET = 'LI202HN0RHIHWIIQQPWSYIPUQE14DLGSNALI21B5XGV4E31Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500

In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:    
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print('\tProblem getting info')
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
dfw_venues = getNearbyVenues(names=dfw['Neighborhood'],
                                   latitudes=dfw['Latitude'],
                                   longitudes=dfw['Longitude']
                                  )
print('There are {} uniques categories.'.format(len(dfw_venues['Venue Category'].unique())))

Briercliff
Fairmount
Willow Creek
Shaw Clarke
Carver Heights
Santa Fe Enclave
Park Place
Harbour Points
Alexandra Meadows
Chapel Creek
Stop Six
Mitchell Boulevard
Jennings - May St. Louis
Hollow Hills
Reunion District
Arlington Heights
Fossil Hill Estates
Overton South
	Problem getting info
Oakridge Terrace
City Center District
Scenic Bluff
Sendera Ranch
Coppell
Diamond Hill - Jarvis
Hamlet
Bluffview
Basswood Park
Sunset Heights South
The Crossing of Fossil Creek
Sylvan Heights West
South Edgewood
Panther Heights
Hulen Heights
Marine Park
Vista Ridge
Basswood Village
Winnetka Heights
Paschal
Lake Country
Worth Heights
Dells District
Burton Hill Trinity Trails
Stone Meadow
Five Mile Creek
Villages of Woodland Springs
Crestline Area
Candle Ridge West
M Streets
Caville
Central Chapel Creek
Westcliff
The Trails of Marine Creek
Sunset Hills
Far East
Park Hill
Glencrest
Emory Place
Alamo Heights
Ridglea Hills
South Meadows Crossing
Chadwick Farms
El Poly Pyramid
Eagle Ranch
Poynter Crossing


In [73]:
dfw_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alamo Heights,21,21,21,21,21,21
Alexandra Meadows,3,3,3,3,3,3
Altemesa East,4,4,4,4,4,4
Arlington Heights,26,26,26,26,26,26
Arts District,37,37,37,37,37,37
Bal Harbour,11,11,11,11,11,11
Basswood Park,16,16,16,16,16,16
Basswood Village,2,2,2,2,2,2
Beechwood Creek,12,12,12,12,12,12


# Analyze Neighborhoods

In [75]:
# one hot encoding
dfw_onehot = pd.get_dummies(dfw_venues[['Venue Category']], prefix="", prefix_sep="")
# There is Neighborhood in the venue categories, so we drop that column
try:
    del dfw_onehot['Neighborhood']
except:
    print('No venues called Neighborhood')
# add neighborhood column back to dataframe
dfw_onehot['Neighborhood'] = dfw_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfw_onehot.columns[-1]] + list(dfw_onehot.columns[:-1])
dfw_onehot = dfw_onehot[fixed_columns]
dfw_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Carpet Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Gym,College Soccer Field,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Courthouse,Coworking Space,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Summer Camp,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll 

In [76]:
dfw_grouped = dfw_onehot.groupby('Neighborhood').mean().reset_index()
dfw_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Carpet Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Basketball Court,College Gym,College Soccer Field,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Courthouse,Coworking Space,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Shop,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Summer Camp,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toll 

# Group most common venues in each neighborhood

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dfw_grouped['Neighborhood']

for ind in np.arange(dfw_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfw_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Heights,Mexican Restaurant,Supermarket,Furniture / Home Store,Nail Salon,Bar,Greek Restaurant,Garden Center,Mobile Phone Shop,Shopping Mall,Bakery
1,Alexandra Meadows,Cajun / Creole Restaurant,Wine Bar,Dive Bar,Yoga Studio,Food & Drink Shop,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
2,Altemesa East,Park,Plaza,Laundry Service,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
3,Arlington Heights,Pizza Place,Food Truck,Sandwich Place,Park,Hookah Bar,Coffee Shop,Beer Garden,New American Restaurant,Middle Eastern Restaurant,Bar
4,Arts District,Hotel,Farmers Market,Southern / Soul Food Restaurant,Coffee Shop,Restaurant,Thai Restaurant,Taco Place,Mexican Restaurant,French Restaurant,Seafood Restaurant


In [99]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

dfw_grouped_clustering = dfw_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfw_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dfw_merged = dfw

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dfw_merged = dfw_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dfw_merged.head() # check the last columns!

,city,Neighborhood,county,state,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Fort Worth,Fairmount,Tarrant,TX,48.116166,-123.468081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,Fort Worth,Willow Creek,Tarrant,TX,29.653767,-95.482293,4.0,Theater,Tennis Court,Hardware Store,Coffee Shop,Dive Bar,Home Service,Flower Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop
254,Fort Worth,Shaw Clarke,Tarrant,TX,30.142715,-94.191578,2.0,Park,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio
303,Fort Worth,Carver Heights,Tarrant,TX,36.898830,-76.228158,4.0,Discount Store,Chinese Restaurant,Supermarket,Convenience Store,Sandwich Place,Food,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market


## identify neighborhoods with invalid cluster labels

In [100]:
idx = pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').isna()
dfw_merged[idx]


,city,Neighborhood,county,state,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Fort Worth,Briercliff,Tarrant,TX,32.647141,-97.430357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Fort Worth,Fairmount,Tarrant,TX,48.116166,-123.468081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,Fort Worth,Overton South,Tarrant,TX,34.127846,-118.257325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1776,Fort Worth,Marine Park,Tarrant,TX,37.699061,-97.454124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4325,Fort Worth,Chadwick Farms,Denton,TX,32.322970,-90.236582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5254,Fort Worth,Western Hills North,Tarrant,TX,39.069829,-84.367030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6364,Fort Worth,Terrace Landing,Tarrant,TX,30.653552,-88.082219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7116,Fort Worth,Bluebonnet Place,Tarrant,TX,42.388705,-71.111651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7208,Fort Worth,West Morningside,Tarrant,TX,33.585928,-86.739815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,Fort Worth,Eastland,Tarrant,TX,36.060565,-80.288971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Map clusters--removing rows with invalid cluster labels and converting cluster labels to int before we map them

In [101]:

# first, we have to drop non-numeric cluster labels and change them to int
dfw_merged = dfw_merged[pd.to_numeric(dfw_merged['Cluster Labels'], errors='coerce').notnull()]
dfw_merged['Cluster Labels'] = dfw_merged['Cluster Labels'].astype('int')
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfw_merged['Latitude'], dfw_merged['Longitude'], dfw_merged['Neighborhood'], dfw_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Arlington are 32.701938999999996, -97.10562379033699.


In [93]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 0, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4815,Fort Worth,Brittany Place,0,Food,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food Court
8379,Fort Worth,Cookes Meadow,0,Food,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food Court


In [94]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 1, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2727,Fort Worth,Stone Meadow,1,Intersection,Gym,Food,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop
3367,Fort Worth,Caville,1,Golf Course,Gym,Yoga Studio,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
6325,Fort Worth,River Trails,1,Business Service,Yoga Studio,Food,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop
10340,Fort Worth,Historic Southside,1,Nightclub,Arts & Crafts Store,Fried Chicken Joint,History Museum,Electronics Store,Gym,Art Gallery,Gym / Fitness Center,Discount Store,Spa
13393,Fort Worth,Ridglea North,1,Beer Garden,Trail,Gym,Gift Shop,Food & Drink Shop,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
15441,Fort Worth,Dorado Ranch,1,Nightclub,Business Service,Concert Hall,Yoga Studio,Food & Drink Shop,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
15514,Dallas,Preston Hollow,1,Nightclub,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
16550,Fort Worth,Harmony,1,Electronics Store,Yoga Studio,Food & Drink Shop,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food Court


In [95]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 2, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
254,Fort Worth,Shaw Clarke,2,Park,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio
574,Fort Worth,Mitchell Boulevard,2,Park,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio
942,Fort Worth,Fossil Hill Estates,2,Park,Automotive Shop,Yoga Studio,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
1930,Fort Worth,Basswood Village,2,Record Shop,Park,French Restaurant,Flea Market,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market
3937,Fort Worth,Far East,2,Park,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio
4387,Fort Worth,El Poly Pyramid,2,Park,Summer Camp,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
4840,Fort Worth,Bomber Heights,2,Park,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio
6618,Fort Worth,Altemesa East,2,Park,Plaza,Laundry Service,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
8237,Fort Worth,Woodhaven,2,Playground,Park,Food,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
10820,Fort Worth,Central Meadowbrook,2,Playground,Park,Trail,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market


In [96]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 3, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1623,Fort Worth,South Edgewood,3,Park,Pool,Beach,Lake,Art Gallery,Fountain,Football Stadium,Food Truck,Food Service,Food Court
3999,Fort Worth,Park Hill,3,Pool,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
7667,Fort Worth,Burchill,3,Art Gallery,Speakeasy,Pool,Water Park,Hotel,Auto Workshop,Food,Filipino Restaurant,Film Studio,Fish & Chips Shop
10151,Fort Worth,Westcliff West,3,Music Venue,Pool,Yoga Studio,Food,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
11019,Fort Worth,Lakewood Addition,3,Pool,Construction & Landscaping,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
11041,Fort Worth,Remington Point,3,Art Gallery,Farmers Market,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
11577,Fort Worth,Glenwood Triangle,3,Cruise,Pool,Lawyer,Park,Yoga Studio,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop
12901,Fort Worth,Crawford Farms,3,Pool,Pizza Place,Soccer Field,Video Game Store,Yoga Studio,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop
13617,Fort Worth,FW Vista West,3,Pool,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
15848,Fort Worth,Fairhaven,3,Pool,Art Gallery,Baseball Field,Lake,Food & Drink Shop,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop


In [97]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 4, dfw_merged.columns[[0,1] + list(range(6, dfw_merged.shape[1]))]]

,city,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
250,Fort Worth,Willow Creek,4,Theater,Tennis Court,Hardware Store,Coffee Shop,Dive Bar,Home Service,Flower Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop
303,Fort Worth,Carver Heights,4,Discount Store,Chinese Restaurant,Supermarket,Convenience Store,Sandwich Place,Food,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market
308,Fort Worth,Santa Fe Enclave,4,Bank,Tapas Restaurant,Grocery Store,History Museum,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
345,Fort Worth,Park Place,4,Gym / Fitness Center,Gym,Hockey Arena,Fried Chicken Joint,Breakfast Spot,Mexican Restaurant,Supermarket,Gift Shop,Sandwich Place,Liquor Store
414,Fort Worth,Harbour Points,4,Brewery,Bar,Sandwich Place,Hotel,Pizza Place,Coffee Shop,Gym,Plaza,Art Gallery,Deli / Bodega
460,Fort Worth,Alexandra Meadows,4,Cajun / Creole Restaurant,Wine Bar,Dive Bar,Yoga Studio,Food & Drink Shop,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
563,Fort Worth,Chapel Creek,4,Furniture / Home Store,Food & Drink Shop,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food Court
564,Fort Worth,Stop Six,4,Liquor Store,Chinese Restaurant,Supermarket,Hookah Bar,Discount Store,Pizza Place,Spa,Park,Outdoor Supply Store,Optical Shop
708,Fort Worth,Jennings - May St. Louis,4,Furniture / Home Store,IT Services,American Restaurant,Convenience Store,Food Court,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
790,Fort Worth,Hollow Hills,4,Bar,Dive Bar,Park,American Restaurant,Bike Rental / Bike Share,Sports Bar,Ice Cream Shop,Food Court,Fish & Chips Shop,Fish Market
